상권수집을 '로' 기준으로 하니 너무 적어서 단속지점 기중 주변 상권으로 설정하려다가
안돼서 그냥 수원시를 300m간격의 grid로 나누고 그 300m둘레의 상권의 개수를 셈


In [11]:
# -*- coding: utf-8 -*-
"""
violations.json 단속지점 좌표를 300m 반경으로 클러스터링 후
클러스터 중심 좌표 기준 상권 개수 집계
출력: 주정차위반기준상권.csv
"""

import os
import time
import json
import pandas as pd
import requests
from tqdm import tqdm
from sklearn.cluster import DBSCAN
import numpy as np

# ================= 설정 =================
API_KEY = os.getenv("KAKAO_API_KEY", "41d56449d3dce5dde7d48ffb261f0e8a")

CATEGORIES = {
    "FD6": "음식점",
    "CE7": "카페",
    "BK9": "은행",
    "HP8": "병원",
    "PO3": "관공서",
    "PK6": "주차장",
}

RADIUS_M = 300   # 반경(m)
SLEEP_SEC = 0.08
HEADERS = {"Authorization": f"KakaoAK {API_KEY}"}

# ================= 유틸 함수 =================
def _request_json(url: str, params: dict, retry: int = 3, backoff: float = 0.6) -> dict:
    last_err = None
    for i in range(retry):
        try:
            res = requests.get(url, headers=HEADERS, params=params, timeout=6)
            if res.status_code == 200:
                return res.json()
            last_err = f"status={res.status_code}, body={res.text[:200]}"
        except Exception as e:
            last_err = str(e)
        time.sleep(backoff * (2 ** i))
    raise RuntimeError(f"GET 실패: {url} params={params} err={last_err}")

def count_nearby(lat: float, lng: float, category_group_code: str, radius_m: int = 300) -> int:
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    params = {
        "category_group_code": category_group_code,
        "x": lng, "y": lat,
        "radius": radius_m,
        "size": 1, "page": 1,
    }
    try:
        data = _request_json(url, params)
        meta = data.get("meta", {})
        return int(meta.get("total_count", 0))
    except Exception as e:
        print(f"❌ 개수 조회 실패: code={category_group_code} ({lat},{lng}) -> {e}")
        return 0

# ================= 메인 로직 =================
def cluster_and_enrich(violations_path="violations.json", out_csv="주정차위반기준상권.csv"):
    with open(violations_path, "r", encoding="utf-8") as f:
        violations = json.load(f)

    # 좌표 배열 (lat, lon)
    coords = np.array([[v["lat"], v["lon"]] for v in violations])

    # DBSCAN: eps≈0.003도 (약 300m), min_samples=1
    clustering = DBSCAN(eps=0.003, min_samples=1, metric="euclidean").fit(coords)
    labels = clustering.labels_

    # DataFrame 생성
    df = pd.DataFrame(violations)
    df["cluster"] = labels

    # 클러스터별 평균 좌표 + 단속건수 합계
    clusters = df.groupby("cluster").agg({
        "lat": "mean",
        "lon": "mean",
        "count": "sum"
    }).reset_index().rename(columns={"lon": "lng", "count": "단속건수합계"})

    print(f"✅ 총 {len(clusters)}개 클러스터 생성됨")

    rows = []
    # ✅ tqdm으로 진행률 표시
    for _, r in tqdm(clusters.iterrows(), total=len(clusters), desc="클러스터 상권 조회중"):
        row = {
            "cluster_id": int(r["cluster"]),
            "lat": r["lat"],
            "lng": r["lng"],
            "단속건수합계": int(r["단속건수합계"]),
        }
        total = 0
        for code, name in CATEGORIES.items():
            cnt = count_nearby(row["lat"], row["lng"], code, RADIUS_M)
            row[name] = cnt
            total += cnt
            time.sleep(SLEEP_SEC)
        row["총합"] = total
        rows.append(row)

    df_out = pd.DataFrame(rows)
    df_out.to_csv(out_csv, index=False, encoding="utf-8-sig")

    print(f"\n📦 저장 완료: {out_csv}")
    print(df_out.head())
    return df_out


if __name__ == "__main__":
    cluster_and_enrich()



✅ 총 42개 클러스터 생성됨


클러스터 상권 조회중: 100%|████████████████████████████████████████████████████████████| 42/42 [00:37<00:00,  1.13it/s]


📦 저장 완료: 주정차위반기준상권.csv
   cluster_id        lat         lng  단속건수합계  음식점  카페  은행  병원  관공서  주차장  총합
0           0  37.272840  127.019846  302922   23  10   7  12    0    4  56
1           1  37.269641  126.952887   19726   45   7   2   1    0    1  56
2           2  37.233443  126.976255     376    6   5   1   2    0    0  14
3           3  37.322786  126.991538      84    1   0   0   0    0    0   1
4           4  37.266418  126.983914     259    5   1   0   0    0    1   7


In [13]:
import pandas as pd

# 1) 파일 이름 지정
counts_csv = "주정차위반기준상권.csv"
counts_json = "facilities.json"

# 2) CSV 불러오기
df_counts = pd.read_csv(counts_csv)

# 3) JSON으로 저장 (UTF-8, 보기 좋게 들여쓰기)
df_counts.to_json(counts_json, orient="records", force_ascii=False, indent=2)

print(f"✅ {counts_json} 저장 완료")

✅ facilities.json 저장 완료


In [17]:
pip install shapely pyproj tqdm requests pandas

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   --------- ------------------------------ 0.4/1.7 MB 12.6 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.7 MB 16.1 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 13.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ----- ---------------------------------- 0.8/6.3 MB 26.2 MB/s eta 0:00:01
   ---------- ----------------------------- 1.6/6.3 MB 20.6 MB/s eta 0:00:01
   ---------------- ----------------------- 2.5/6.3 MB 20.2 MB/s eta 0:00:01
   --------------------- ------------------ 3.4/6.3 MB 21.5 MB/s eta 0:00:01
   ------------------------- -------------- 4.0/6.3 MB 19.7 MB/s eta 0:00:01
   ------------------------------ --------- 4.8/6.3 MB 19.1 MB/s eta 0:00:01
   ----------------------------------- ---- 5.6/6.3 MB 19.9 MB/s eta 0:00:01
   ---------------------------------------  6.3/6.3 MB 19.2 MB/s eta 0:00:01
   ------------

이거로 facilities저장함

In [8]:
# -*- coding: utf-8 -*-
"""
수원시 bbox 영역을 일정 간격(Grid)으로 샘플링하여
각 그리드 포인트 반경 내 카카오 카테고리 개수를 집계
출력: grid_poi_suwon.csv
"""
import os, math, time, json, requests
import pandas as pd
from tqdm import tqdm

# ===== 설정 =====
API_KEY = os.getenv("KAKAO_API_KEY", "41d56449d3dce5dde7d48ffb261f0e8a")
HEADERS = {"Authorization": f"KakaoAK {API_KEY}"}

# 수원시 대략 bbox (남,서,북,동)
SUWON_BBOX = (37.20, 126.90, 37.35, 127.10)

GRID_STEP_M = 300     # 격자 간격 (m) — 200~600 사이에서 조절
RADIUS_M    = 300     # 카테고리 검색 반경 (m)
SLEEP_SEC   = 0.08    # rate limit 완화
VERIFY_CITY = False   # True면 역지오코딩으로 '수원시'만 남김(호출량 증가)
CACHE_PATH  = "kakao_count_cache.json"

CATEGORIES = {
    "FD6":"음식점",
    "CE7":"카페",
    "BK9":"은행",
    "HP8":"병원",
    "PO3":"관공서",
    "PK6":"주차장",
}

# ===== 유틸 =====
def load_cache():
    if os.path.exists(CACHE_PATH):
        try:
            return json.load(open(CACHE_PATH, "r", encoding="utf-8"))
        except Exception:
            return {}
    return {}

def save_cache(cache):
    tmp = CACHE_PATH + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(cache, f, ensure_ascii=False)
    os.replace(tmp, CACHE_PATH)

def cache_key(lat,lng,code,r):
    return f"{round(lat,6)}|{round(lng,6)}|{code}|{r}"

def _get(url, params, retry=3, backoff=0.6):
    last = None
    for i in range(retry):
        try:
            r = requests.get(url, headers=HEADERS, params=params, timeout=8)
            if r.status_code == 200:
                return r.json()
            last = f"status={r.status_code} body={r.text[:120]}"
        except Exception as e:
            last = str(e)
        time.sleep(backoff*(2**i))
    raise RuntimeError(last)

def kakao_total_count(lat,lng,code,rad,cache):
    k = cache_key(lat,lng,code,rad)
    if k in cache:
        return cache[k]
    data = _get("https://dapi.kakao.com/v2/local/search/category.json",
                {"category_group_code":code,"x":lng,"y":lat,"radius":rad,"size":1,"page":1})
    total = int(data.get("meta",{}).get("total_count",0))
    cache[k] = total
    return total

def kakao_region(lat,lng):
    data = _get("https://dapi.kakao.com/v2/local/geo/coord2address.json", {"x":lng,"y":lat})
    docs = data.get("documents",[])
    if not docs: return None
    return docs[0].get("road_address") or docs[0].get("address")

def meters_to_deg(lat, meters):
    # 위도 1도 ≈ 111,320 m
    dlat = meters / 111320.0
    # 경도 1도 ≈ 111,320 * cos(lat)
    dlon = meters / (111320.0 * math.cos(math.radians(lat)))
    return dlat, dlon

def grid_points(bbox, step_m):
    S,W,N,E = bbox
    mid_lat = (S+N)/2
    dlat, dlon = meters_to_deg(mid_lat, step_m)
    pts = []
    lat = S
    while lat <= N:
        lon = W
        while lon <= E:
            pts.append((round(lat,6), round(lon,6)))
            lon += dlon
        lat += dlat
    return pts

# ===== 메인 =====
def build_grid(out_csv="grid_poi_suwon.csv"):
    cache = load_cache()
    pts = grid_points(SUWON_BBOX, GRID_STEP_M)
    print(f"그리드 점 개수: {len(pts)}")

    rows = []
    for (lat,lng) in tqdm(pts, desc="카테고리 집계"):
        if VERIFY_CITY:
            info = kakao_region(lat,lng)
            if not info: 
                time.sleep(SLEEP_SEC); 
                continue
            # road_address 또는 address의 2depth가 '수원시'인지 확인
            if not (info.get("region_2depth_name") == "수원시" or info.get("region_2depth_name") == "수원특례시"):
                time.sleep(SLEEP_SEC); 
                continue

        row = {"lat":lat, "lng":lng}
        total = 0
        for code,label in CATEGORIES.items():
            cnt = kakao_total_count(lat,lng,code,RADIUS_M,cache)
            row[label] = cnt
            total += cnt
            time.sleep(SLEEP_SEC)
        row["총합"] = total
        rows.append(row)

    save_cache(cache)

    if not rows:
        print("⚠️ 결과가 비었습니다.")
        return None

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"📦 저장 완료: {out_csv}")
    print(df.head())
    return df

if __name__ == "__main__":
    build_grid()


그리드 점 개수: 3360


카테고리 집계: 100%|███████████████████████████████████████████████████████████████| 3360/3360 [51:58<00:00,  1.08it/s]

📦 저장 완료: grid_poi_suwon.csv
    lat         lng  음식점  카페  은행  병원  관공서  주차장  총합
0  37.2  126.900000    3   0   0   0    0    0   3
1  37.2  126.903387    0   0   0   0    0    0   0
2  37.2  126.906773    0   0   0   0    0    0   0
3  37.2  126.910160    1   1   0   0    0    0   2
4  37.2  126.913547    1   1   0   0    0    0   2


In [10]:
import pandas as pd

# 1) 파일 이름 지정
counts_csv = "grid_poi_suwon.csv"
counts_json = "facilities.json"

# 2) CSV 불러오기
df_counts = pd.read_csv(counts_csv)

# 3) JSON으로 저장 (UTF-8, 보기 좋게 들여쓰기)
df_counts.to_json(counts_json, orient="records", force_ascii=False, indent=2)

print(f"✅ {counts_json} 저장 완료")

✅ facilities.json 저장 완료


In [1]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("grid_poi_suwon.csv")

# 각 열의 최소값과 최대값 계산
min_values = df.min()
max_values = df.max()

print("최솟값:")
print(min_values)
print("\n최댓값:")
print(max_values)


최솟값:
lat     37.2
lng    126.9
음식점      0.0
카페       0.0
은행       0.0
병원       0.0
관공서      0.0
주차장      0.0
총합       0.0
dtype: float64

최댓값:
lat     37.348221
lng    127.099816
음식점    462.000000
카페     118.000000
은행      28.000000
병원      98.000000
관공서      6.000000
주차장     45.000000
총합     716.000000
dtype: float64
